In [1]:
import logging
import pymongo
import psycopg2

In [6]:
logging.basicConfig(level=logging.INFO, filename="movements.log")
logger = logging.getLogger('movements_mongo_to_psql')

In [3]:
mongo_database  = pymongo.MongoClient()["scraper2"]
mongo_movements = mongo_database["container_movements"]

In [ ]:
dsn = {
    "host": "HOST",
    "dbname": "tracking",
    "user": "postgres",
    "password": "PASSWORD"
}

In [4]:
def get_vehicle(movement):
    vehicle = movement.get("vehicle")
    if vehicle.lower() == "vessel":
        return 1
    if vehicle.lower() == "truck":
        return 2
    if vehicle.lower() == "train":
        return 3
    if vehicle.lower() == "unknown":
        return None
    logger.info('Vehicle "{}" was not found'.format(vehicle))
    return None

def get_carrier(movement):
    carrier = movement.get("carrier")
    if carrier.lower() == "maersk":
        return 1
    if carrier.lower() == "hapag-lloyd":
        return 2
    if carrier.lower() == "evergreen":
        return 3
    raise Exception("Could not find carrier {}".format(carrier))

def get_status(movement, cur, carrier):
    status = movement.get("status")
    # Validar el estado
    if (status is None) or (len(status) <= 0):
        raise Exception("Stauts is empty")
    # Obtener el estado
    cur.execute("SELECT id FROM tracking_movement_status WHERE name = %s AND enterprise_id = %s;", (status, carrier))
    result = cur.fetchone()
    if result is not None:
        return result[0]
    # Insertar estado en base de datos
    cur.execute("INSERT INTO tracking_movement_status (status, name, enterprise_id) VALUES (%s, %s, %s) RETURNING id;",
                (0, status, carrier))
    result = cur.fetchone()
    logger.info('Status "{}" created for carrier {} with value 0'.format(status, carrier))
    return result[0]

def get_location(movement, cur):
    # Validar los datos de ubicación
    location  = movement.get("location")
    if (location is None) or (len(location) <= 0):
        raise Exception("Location is empty")
    location  = location.split("\n")[-1]
    latitude  = movement.get("latitude")
    longitude = movement.get("longitude")
    if (latitude is None) or (longitude is None):
        logger.info('Geolocation data for location "{}" is empty'.format(location))
    # Obtener la ubicación
    cur.execute("SELECT id FROM tracking_location WHERE name = (%s);", (location,))
    result = cur.fetchone()
    if result is not None:
        return result[0]
    # Si es que no se encontró, insertar ubicación en base de datos
    cur.execute("INSERT INTO tracking_location (name, latitude, longitude) VALUES (%s, %s, %s) RETURNING id;",
                (location, latitude, longitude))
    result = cur.fetchone()
    logger.info('Location "{}" created with latitude {} and longitude {}'.format(location, latitude, longitude))
    return result[0]

In [8]:
with psycopg2.connect(**dsn) as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT id, code, carrier_id FROM tracking_container;")
        containers = cur.fetchall()
        for id, code, carrier in containers:
            query = {"container": code}
            sorting = [('_id', pymongo.ASCENDING), ('date', pymongo.ASCENDING)]
            movements = mongo_movements.find(query).sort(sorting)
            for movement in movements:
                data = {
                    "container": id,
                    "location": get_location(movement, cur),
                    "status": get_status(movement, cur, carrier),
                    "date": movement.get("date"),
                    "vehicle": get_vehicle(movement),
                    "vessel": movement.get("vessel"),
                    "voyage": movement.get("voyage"),
                    "estimated": movement.get("estimated"),
                    "created_at": movement.get("created_at")
                }
                cur.execute("""
                INSERT INTO tracking_movement (container_id, location_id, status_id, date, vehicle_id, vessel, voyage,
                                                estimated, created_at)
                VALUES (%(container)s, %(location)s, %(status)s, %(date)s, %(vehicle)s, %(vessel)s, %(voyage)s,
                                                %(estimated)s, %(created_at)s)
                """, data)
    conn.commit()
conn.close()